In [1]:
import   pandas as pd
import os
import gurobipy as gp
from gurobipy import GRB

In [2]:
def conjuntos():
    
    # K: Conjunto de Navios
    K  = [0, 1, 2, 3]

    # N: Conjunto de Tarefas 
    N = [0, 1, 2, 3, 4, 5, 6, 7 ,8, 9, 10 ,11, 12, 13, 14, 15, 16]

    # N_j: Conjunto de navios (k) compatíveis com a tarefa j 
    K_j = [[0, 1, 2, 3],
           [0, 1],
           [0, 2],
           [3],
           [1, 3],
           [1, 3],
           [1, 2],  
           [3],
           [0, 2, 3],
           [0, 1, 3],
           [2, 3],
           [0, 1],
           [0, 2],
           [1, 2],
           [0],
           [2],
           [0, 1, 2, 3]]
    
    # K_k: Conjunto de tarefas (j=índice da tabela) compatíveis com o navio k (cada linha é um navio)
    K_k = [[0,  1,  2,  8,  9, 11, 12, 14, 16],
           [0,  1,  4,  5,  6,  9, 11, 13, 16],
           [0,  2,  6,  8, 10, 12, 13, 15, 16],
           [0,  3,  4,  5,  7,  8,  9, 10, 16]]


    return K, N, K_j

In [3]:
# PARAMETROS

def puxa_dados(nome_arq, aba):
    return pd.read_excel(nome_arq, sheet_name=aba) 

def parametros():
    
    aj = [0]*14 + [3] + [7.5] + [0]             #Limite inferior - janela de tempo
    bj = [12] + [12]*13 + [3.1] + [7.6] + [12]  #Limite superior - janela de tempo

    # wj: Penalidade Linear pelo atraso do início da tarefa j 
    wj = [0, 8000, 0, 8000, 13000, 15000, 7000, 12000, 6000, 14000, 5000, 6000, 2000, 0, 0, 0, 0]

    # tj: duração média da realização da tarefa j
    tj = [0, 0.5, 2.9, 1.6, 1.8, 1.6, 0.7, 1.6, 1, 2.2, 0.8, 3, 0.6, 2.9, 0.5, 0.5, 0]
    
    # rj: Instante de liberação da tarefa (para curto praço = 0)
    rj = [0]*16
    
    # rk: Tempo de liberação da embarcação k
    rk = [0.3, 0.4, 0.7, 0.1]
    
    
    #  Puxa os dados - Excel
    path = os.getcwd()
    nome_arq = 'Dados_otimiza.xlsx'

    t_ij1 = puxa_dados(nome_arq, 'TDNAV01')  #Tempo de Deslocamento do Navio 01 pelo arco ij
    t_ij2 = puxa_dados(nome_arq, 'TDNAV02')  #Tempo de Deslocamento do Navio 02 pelo arco ij
    t_ij3 = puxa_dados(nome_arq, 'TDNAV03')  #Tempo de Deslocamento do Navio 03 pelo arco ij
    t_ij4 = puxa_dados(nome_arq, 'TDNAV04')  #Tempo de Deslocamento do Navio 04 pelo arco ij
    t_ijk = [t_ij1, t_ij2, t_ij3, t_ij4]           #Matriz com todas as informações de tempo de deslocamento
    
    # c_ijk: Custo de deslocamento do navio k pelo arco ij
    c_ijk = [t_ij1*20*1000, t_ij2*20*1000, t_ij3*30*1000, t_ij4*30*1000]
    
    return aj, bj, wj, tj, rj, rk, t_ijk, c_ijk

In [6]:
# MAIN

K, N, K_j = conjuntos()
aj, bj, wj, tj, rj, rk, t_ijk, c_ijk = parametros()

###################   Definição do Modelo
m = gp.Model("teste")


"""         VARIÁVEIS DE DECISÃO         """

###################   Variável de Decissão 01 (x)
# X: vetor com uma matriz para cada navio k
# Cada matriz[k] representa a variável 1 se o navio percorre o arco ij, e 0 caso contário
# Variável Binária
x =[]
for k in K:
    x.append([])
    for i in N:
        x[k].append([])
        for j in N:
            x[k][i].append(m.addVar(vtype=GRB.BINARY, name =f"x_{k}_{i}_{j}"))

###################   Variável de Decissão 02 (s)
# S: Vetor com uma lista para cada navio k     
# Cada lista[k] possui o instante de início da tareja j    
# Variável contínua  
s = []
for k in K:
    s.append([])
    for j in N:
        s[k].append(m.addVar(lb=0.0, vtype=GRB.CONTINUOUS, name =f"s_{k}_{j}"))
        

"""         RESTRIÇÕES         """

###################   Restrição 01
#Força que o roteiro de cada navio comece na posição inicial do mesmo
#Na matriz de variáveis vinária (x_ijk), a soma de todas as colunas da linha 0 vale 1
for k in K:
    aux = 0
    #for j in N:
    for j in N[1:len(N)-1]:
        aux += x[k][0][j]   
    #m.addConstr(aux == 1, f"c01_{k}_{j}")
    m.addConstr(aux == 1, f"c01_{k}")
    

###################   Restrição 02
#Impõe a conservação do fluxo em cada nó
#Toda "coluna" da matriz Xijk deve ter alguma linha com o valor "1"
for k in K:
    for j in N[1:len(N)-1]:        
        aux1 = 0
        aux2 = 0
        #for i in N:
        for i in N[0:len(N)-1]:
            if i!=j:
                aux1 += x[k][i][j]
        for i in N[1:]:
            if i!=j:
                aux2 += x[k][j][i]
        m.addConstr(aux1-aux2 == 0, f"c02_{k}_{j}")
    
###################   Restrição 03
#Garante que a tarefa n seja realizada por uma embarcação compatível(K_j[j])
for j in N[1:len(N)-1]:
    aux = 0
    for k in K_j[j]:
        #for i in N:
        for i in N[0:len(N)-1]:
            if i!= j:
                aux += x[k][i][j]            
    #m.addConstr(aux == 1, f"c03_{k}_{j}")
    m.addConstr(aux == 1, f"c03_{j}")

###################   Restrição 04
#Controla o instante de chegada de cada tarefa
#Impõe que o instante de chegada seja maior que o a soma(tempo de liberação da atividade anterior + tempo de deslocamento)
for k in K:
    for i in N:
        for j in N:            
            aux = 0
            #T = bj[i]-aj[j]
            T = 12
            #aux = s[k][i] + t_ijk[k].values[i][j] - ((1-x[k][i][j])*T)
            aux = s[k][j] - s[k][i] - tj[i] - t_ijk[k].values[i][j] + ((1-x[k][i][j])*T)
            #m.addConstr(s[k][j] >= aux, f"c04_{k}_{i}_{j}")
            m.addConstr(aux >= 0, f"c04_{k}_{i}_{j}")
            

###################   Restrição 05
#Impõe as janelas de tempo para as tarefas
#O instante de chegada do navio k na tarefa j precisa estar entre aj e bj
for k in K:
    for j in N[1:len(N)-1]:
        aux = 0        
        #for i in N:
        for i in N[0:len(N)-1]:
            if i!=j:
                aux += x[k][i][j]    
        m.addConstr(s[k][j] >= aux*aj[j], f"c05_INF{k}_{j}")
        m.addConstr(s[k][j] <= aux*bj[j], f"c05_SUP{k}_{j}")
 
###################   Restrição 06
#Impõe os instante de liberação da frota
for k in K:
    #for j in N:
    for j in N[1:len(N)-1]:
        aux = x[k][0][j]*(rk[k] + t_ijk[k].values[0][j] )
        m.addConstr(s[k][j] >= aux, f"c06_LIB{k}_{j}")          
  
    
###################   Restrição 07
#Apenas impõe que todas variáveis s[k][j] sejam maiores/iguais a zero
# desnecessário... ver a definição da variável (foi adicionado um lower bound lb = 0.0)
#for k in K:
#    for j in N:
#        m.addConstr(s[k][j] >= 0, f"c07_ESP{k}{j}") 


# Zera fluxos inconsistentes
aux = 0
for k in K:
    for i in N:
        for j in N:
            if i==j or i==len(N)-1:
                aux += x[k][i][j]
m.addConstr(aux == 0, "c07")          


"""         FUNÇÃO OBJETIVO         """

exp1 = 0
exp2 = 0
for k in K:
    #for i in N:
    for i in N[1:len(N)-1]:
        for j in N[1:]:
            if i!=j:
                exp1 += x[k][i][j]*c_ijk[k].values[i][j]
    
    for j in N[1:len(N)-1]:
        exp2 += s[k][j]*wj[j]
            
m.setObjective(exp1 + exp2, GRB.MINIMIZE)

#m.computeIIS()
m.write("model.lp")
m.setParam("TimeLimit", 7200.0);  # rodar pelo menos 15 minutos (900 segundos) -> o ideal é rodar por 1 hora (3600)
m.optimize()
m.write("solucao.sol")

# Imprime as variáveis de decisão não nulas
for k in K:
    for i in N:
        for j in N:
            if x[k][i][j].X>0.01:
                print(f"{x[k][i][j].VarName} = {x[k][i][j].X}")

for k in K:
    for i in N:
        if s[k][i].X>0.01:
            print(f"{s[k][i].VarName} = {s[k][i].X}")

            

Set parameter TimeLimit to value 7200
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (mac64[x86])
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1416 rows, 1224 columns and 7004 nonzeros
Model fingerprint: 0xda6c84a7
Variable types: 68 continuous, 1156 integer (1156 binary)
Coefficient statistics:
  Matrix range     [4e-01, 1e+01]
  Objective range  [8e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 145 rows and 202 columns
Presolve time: 0.11s
Presolved: 1271 rows, 1022 columns, 12148 nonzeros
Variable types: 68 continuous, 954 integer (954 binary)

Root relaxation: objective 5.167989e+04, 194 iterations, 0.04 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 51679.8854    0   39          - 51679.8854      -     -    0s
     0     0 93725.8722    0